In [1]:
import pandas as pd

In [2]:
df =pd.read_csv('../data/data.csv',usecols=lambda x: x != "id")

In [3]:
!wget https://github.com/alexeygrigorev/minsearch/blob/main/minsearch/minsearch.py

--2025-08-07 14:43:31--  https://github.com/alexeygrigorev/minsearch/blob/main/minsearch/minsearch.py
Resolving github.com (github.com)... 20.26.156.215
Connecting to github.com (github.com)|20.26.156.215|:443... connected.
200 OKequest sent, awaiting response... 
Length: unspecified [text/html]
Saving to: ‘minsearch.py.1’

minsearch.py.1          [ <=>                ] 190.30K  --.-KB/s    in 0.002s  

2025-08-07 14:43:31 (99.3 MB/s) - ‘minsearch.py.1’ saved [194869]



In [4]:
df.columns

Index(['exercise_name', 'type_of_activity', 'type_of_equipment', 'body_part',
       'type', 'muscle_groups_activated', 'instructions'],
      dtype='object')

In [5]:
!more minserach.py

more: cannot open minserach.py: No such file or directory


In [6]:
ls

RAG_test.ipynb  minsearch.py  minsearch.py.1


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [8]:
model_name = "Qwen/Qwen3-1.7B"

In [9]:
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]

In [11]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)


In [13]:
messages

[{'role': 'user',
  'content': 'Give me a short introduction to large language model.'}]

In [14]:
model_inputs

{'input_ids': tensor([[151644,    872,    198,  35127,    752,    264,   2805,  16800,    311,
           3460,   4128,   1614,     13, 151645,    198, 151644,  77091,    198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [15]:
# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 


In [22]:
# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")


In [23]:
print("thinking content:", thinking_content)
print("content:", content)

thinking content: <think>
Okay, the user wants a short introduction to large language models. Let me start by defining what they are. Large language models are AI systems that can understand and generate human-like text. I should mention their key features like training on vast datasets, neural networks, and the ability to generate text on the fly.

Wait, they might not know the specifics. I need to keep it simple. Maybe start with the basics: what they are, how they work, and their applications. Also, highlight their capabilities like language understanding, generation, and reasoning. Should I mention specific examples like GPT or BERT? Probably not necessary for a short intro. Focus on the general concept.

Make sure to explain that they're trained on a lot of data, which allows them to learn patterns and generate coherent text. Also, note that they can be used in various fields like writing, coding, or customer service. Avoid jargon but keep it informative. Check for clarity and con